<a href="https://colab.research.google.com/github/NoahIslam/DeepLearningSpectrogramModels/blob/main/ViT_Transformer_SpectrogramDeliriumClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'EDF Files'
assert FOLDERNAME is not None, "[!] Enter the foldername."
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyedflib
!pip install datasets transformers
from datetime import datetime
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split, WeightedRandomSampler
from torchvision import transforms
import torch.nn.functional as F
from PIL import Image
import pandas as pd
import pickle
import pyedflib
import random

USE_GPU = True
dtype = torch.float32

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print_every = 5
print('using device:', device)

plt.ioff()

In [ ]:
def get_list_of_files(filepath):
  entries = os.listdir(filepath)
  files = []
  for item in entries:
    if os.path.isdir(os.path.join(filepath, item)):
      files.extend(get_list_of_files(os.path.join(filepath, item)))
  files.extend([os.path.join(filepath, x) for x in os.listdir(filepath) if os.path.isfile(os.path.join(filepath, item))])
  return files

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/EEG_Clinical_Data_Master_File_Broadcasted.xlsx')
df.to_csv('EEG_Data.csv', index=False)

In [ ]:
def to_datetime(timedate):
  rem_paren = timedate.split('/')[-1]
  filename = rem_paren.split("_")
  EEG_ID = filename[0]
  date = filename[1]
  if date[len(date)-4:len(date)] == '2019' or date[len(date)-4:len(date)] == '2020' or date[len(date)-4:len(date)] == '2018' or date[len(date)-4:len(date)] == '2017' or date[len(date)-4:len(date)] == '2016':
    month = date[len(date)-8:len(date)-6]
    day = date[len(date)-6:len(date)-4]
    year = date[len(date)-4:len(date)]
  else:
    year = date[len(date)-8:len(date)-4]
    month = date[len(date)-4:len(date)-2]
    day = date[len(date)-2:len(date)]
  try:
    return (int(EEG_ID), datetime(int(year), int(month), int(day), 0, 0))
  except Exception as e:
    return ()

def csv_to_dict():
  folder_path = '/content/drive/MyDrive/EDF Files/'
  filename_to_score = {}
  filename_to_spec = {}
  files = get_list_of_files(folder_path)
  shitlist = []
  not_edf = ['/content/drive/MyDrive/EDF Files/August 2019 Lettered/2090_CK20190809_1228_A.edf', '/content/drive/MyDrive/EDF Files/August 2019 Lettered/2087_JC20190807_1015_A.edf', '/content/drive/MyDrive/EDF Files/August 2019 Lettered/2101_RC20190828_1140_A.edf', '/content/drive/MyDrive/EDF Files/July 2019 Numbered/2071_MM20190723_1013_3.edf', '/content/drive/MyDrive/EDF Files/July 2019 Numbered/2077_JJ20190729_1253_3.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/July 2017 EDF files/week of 17Jul10 - 17Jul26/0551_KR20170711_1016-1026.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/Jun 2017 EDF files/week of 6.7.17 - 6.12.17/0504_DT20170607_1130-1140.DAT', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/May 2017 EDF files/0475_20170523_1623-1633.DAT', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr14 - 18Apr20/1185_TS20180419_1315-1320.DAT', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr14 - 18Apr20/1189_LH20180419_1330-1335.DAT', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr14 - 18Apr20/1179_MH20180419_1410-1415.DAT', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr14 - 18Apr20/1177_SM20180419_1350-1355.DAT', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr14 - 18Apr20/1195_JK20180420_1430-1435.DAT', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr14 - 18Apr20/1184_GR20180420_1435-1440.DAT', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr14 - 18Apr20/1185_TS20180420_1530-1535.DAT', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr14 - 18Apr20/1179_MH20180420_1450-1455.DAT', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/Sep 2017 EDF files/week of 17Sep7 - 17Sep14/0714_RC20170914_0924-0931.DAT', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/Sep 2017 EDF files/week of 17Sep7 - 17Sep14/0716_DA20170914_1035-1040.DAT', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/Sep 2017 EDF files/week of 17Sep7 - 17Sep14/0710_BK20170914_1123-1128.DAT']
  # for length of 140
  too_short = ['/content/drive/MyDrive/EDF Files/November 2018/1561_RW20181130_0955-0956.edf', '/content/drive/MyDrive/EDF Files/November 2018/1562_SG20181130_1019-1020.edf', '/content/drive/MyDrive/EDF Files/November 2018/1561_RW20181130_1416.edf', '/content/drive/MyDrive/EDF Files/November 2018/1562_SG20181130_1440.edf', '/content/drive/MyDrive/EDF Files/January 2020 Lettered/2351_FS01272020_1118_A.edf', '/content/drive/MyDrive/EDF Files/October 2019 Numbered/2200_BF20191004_1020_2.edf', '/content/drive/MyDrive/EDF Files/October 2019 Numbered/2227_TM20191021_1046_2.edf', '/content/drive/MyDrive/EDF Files/May 2019 Lettered/1989_WH20190516_1147-1150_B.edf', '/content/drive/MyDrive/EDF Files/December 2018/1561_RW20181203_stabilizing.edf', '/content/drive/MyDrive/EDF Files/December 2018/1568_LW20181203_stabilizing.edf', '/content/drive/MyDrive/EDF Files/August 2019 Numbered/2093_BL08142019_1419_3.edf', '/content/drive/MyDrive/EDF Files/September 2019 Numbered/2165_LP20190920_1027_3.edf', '/content/drive/MyDrive/EDF Files/January 2019/1592_JE20190110_1053-1055.edf', '/content/drive/MyDrive/EDF Files/January 2019/1601_LS20190118_1003-1006.edf', '/content/drive/MyDrive/EDF Files/February 2019/1667_MM20190223_1031-1034.edf', '/content/drive/MyDrive/EDF Files/February 2019/1710_CG20190226_1609-1611.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1832_CP20190402_0849-0850.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1846_MS20190402_0923-0924.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1851_WM20190403_0858-0859.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1850_LH20190403_0949-0951.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1850_LH20190403_0949-0951(1).edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1852_DI20190403_1455-1456.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1865_BK20190404_0917-0918(1).edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1865_BK20190404_0917-0918.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1863_DH20190404_1001-1002.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1876_JE20190405_1455-1456.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1827_DS20190406_0941-0942.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1831_DE20190406_1104-1105.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1865_BK20190406_1121-1122.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1886_DD20190409_0835-0837.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1827_DS20190409_0903-0904.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1893_RW20190410_0909-0910.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1891_CT20190410_0947-0947.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1894_RD20190410_1448-1449.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1908_DC20190412_1417-1418.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1888_RK20190413_0914-0915.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1908_DC20190413_0929-0930.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1907_TC20190413_0954-0955.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1905_LS20190413_1024-1025.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1891_CT20190413_1037-1038.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1891_CT20190416_1004-1005.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1905_LS20190416_1019-1020.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1876_JE20190416_1039-1040.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1910_RK20190416_1056-1057.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1911_RR20190416_1440-1441.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1914_DB20190416_1601-1602.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1918_PK20190418_0958-0959_1.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1912_SC20190423_1519_1.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1876_JE20190423_1541_1.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1935_MB20190423_1556_1.edf', '/content/drive/MyDrive/EDF Files/April 2019 Numbered/1960_RK20190425_1549_2.edf', '/content/drive/MyDrive/EDF Files/March 2019/1692_DP20190302_1030-1031.edf', '/content/drive/MyDrive/EDF Files/March 2019/1728_JS20190302_1044-1045.edf', '/content/drive/MyDrive/EDF Files/March 2019/1716_JR20190302_1058-1059.edf', '/content/drive/MyDrive/EDF Files/March 2019/1726_SB20190302_1132-1133.edf', '/content/drive/MyDrive/EDF Files/March 2019/1712_GK20190302_1149-1150.edf', '/content/drive/MyDrive/EDF Files/March 2019/1666_RC20190302_1214-1215.edf', '/content/drive/MyDrive/EDF Files/March 2019/1740_JR20190304_1331-1332.edf', '/content/drive/MyDrive/EDF Files/March 2019/1748_MS20190306_0921-0922.edf', '/content/drive/MyDrive/EDF Files/March 2019/1747_MT20190306_0955-0956.edf', '/content/drive/MyDrive/EDF Files/March 2019/1752_PM20190306_1431-1432.edf', '/content/drive/MyDrive/EDF Files/March 2019/1759_LB20190307_0858-0859.edf', '/content/drive/MyDrive/EDF Files/March 2019/1761_GK20190307_0939-0940.edf', '/content/drive/MyDrive/EDF Files/March 2019/1762_KT20190307_1421-1422.edf', '/content/drive/MyDrive/EDF Files/March 2019/1726_SB20190308_1157-1158.edf', '/content/drive/MyDrive/EDF Files/March 2019/1768_WW20190308_1345-1346.edf', '/content/drive/MyDrive/EDF Files/March 2019/1762_KT20190308_1358-1359.edf', '/content/drive/MyDrive/EDF Files/March 2019/1746_MM20190308_1422-1423.edf', '/content/drive/MyDrive/EDF Files/March 2019/1742_RF20190308_1441-1442.edf', '/content/drive/MyDrive/EDF Files/March 2019/1772_CZ20190312_0923-0924.edf', '/content/drive/MyDrive/EDF Files/March 2019/1766_WS20190312_0955-0956.edf', '/content/drive/MyDrive/EDF Files/March 2019/1766_WS20190312_0956-0958.edf', '/content/drive/MyDrive/EDF Files/March 2019/1742_RF20190312_1009-1010.edf', '/content/drive/MyDrive/EDF Files/March 2019/1761_GK20190312_1029-1030.edf', '/content/drive/MyDrive/EDF Files/March 2019/1752_PM20190312_1046-1047.edf', '/content/drive/MyDrive/EDF Files/March 2019/1776_KW20190312_1508-1509.edf', '/content/drive/MyDrive/EDF Files/March 2019/1780_MD20190313_0925-0926.edf', '/content/drive/MyDrive/EDF Files/March 2019/1742_RF20190313_0946-0947.edf', '/content/drive/MyDrive/EDF Files/March 2019/1766_WS20190313_1003-1004.edf', '/content/drive/MyDrive/EDF Files/March 2019/1776_KW20190313_1014-1015.edf', '/content/drive/MyDrive/EDF Files/March 2019/1782_SD20190313_1437-1439.edf', '/content/drive/MyDrive/EDF Files/March 2019/1786_SS20190313_1500-1501.edf', '/content/drive/MyDrive/EDF Files/March 2019/1789_KT20190314_0911-0912.edf', '/content/drive/MyDrive/EDF Files/March 2019/1790_KN20190314_1353-1353.edf', '/content/drive/MyDrive/EDF Files/March 2019/1752_PM20190315_1215-1216.edf', '/content/drive/MyDrive/EDF Files/March 2019/1784_CR20190315_1243-1244.edf', '/content/drive/MyDrive/EDF Files/March 2019/1761_GK20190315_1303-1304.edf', '/content/drive/MyDrive/EDF Files/March 2019/1793_DD20190315_1318-1319.edf', '/content/drive/MyDrive/EDF Files/March 2019/1787_PL20190315_1328-1329.edf', '/content/drive/MyDrive/EDF Files/March 2019/1786_SS20190315_1431-1432.edf', '/content/drive/MyDrive/EDF Files/March 2019/1782_SD20190315_1500-1501.edf', '/content/drive/MyDrive/EDF Files/March 2019/1797_DR20190325_1345-1346.edf', '/content/drive/MyDrive/EDF Files/March 2019/1798_KG20190325_1411-1412.edf', '/content/drive/MyDrive/EDF Files/March 2019/1798_KG20190326_1439-1440.edf', '/content/drive/MyDrive/EDF Files/March 2019/1794_VK20190326_1542-1543.edf', '/content/drive/MyDrive/EDF Files/March 2019/1819_GN20190327_0928-0929.edf', '/content/drive/MyDrive/EDF Files/March 2019/1817_CH20190327_1402-1403.edf', '/content/drive/MyDrive/EDF Files/March 2019/1807_JL20190328_0852-0853.edf', '/content/drive/MyDrive/EDF Files/March 2019/1827_DS20190328_0903-0904.edf', '/content/drive/MyDrive/EDF Files/March 2019/1827_DS20190328_0904-0907.edf', '/content/drive/MyDrive/EDF Files/March 2019/1830_LS20190328_1447-1447.edf', '/content/drive/MyDrive/EDF Files/March 2019/1830_LS20190329_1235-1236.edf', '/content/drive/MyDrive/EDF Files/March 2019/1827_DS20190329_1303-1304.edf', '/content/drive/MyDrive/EDF Files/March 2019/1835_DH20190329_1340-1341.edf', '/content/drive/MyDrive/EDF Files/March 2019/1807_JL20190329_1343-1344.edf', '/content/drive/MyDrive/EDF Files/March 2019/1832_CP20190402_0849-0850.edf', '/content/drive/MyDrive/EDF Files/March 2019/1827_DS20190402_0908-0909.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18June16-18June22/1331_SA20180619_1522-1524.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/November 2018/1559_BS20181119_1035-1040.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/November 2018/1560_GM20181127_1509-1512.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/Sept 2016 EDF files/0255_20160915_1042-1054.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Mar16 - 18Mar23/1074_MK20180309_1535-1440.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Mar16 - 18Mar23/1099_DN20180319_1315-1320.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Mar16 - 18Mar23/1114_DM20180323_1335-1340.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18oct22 - 18oct31/1511_IF20181022_1012-1017.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18oct22 - 18oct31/1515_SE20181022_1449-1454.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18oct22 - 18oct31/1519_DB20181022_1340-1345.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18oct22 - 18oct31/1514_JS20181022_1426-1430.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 17Dec14 - 18Jan4/0915_JM20171228_1400-1403.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/Jun 2017 EDF files/week of 6.7.17 - 6.12.17/0515_20170610_1236-1246.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18July15-18July20/1373_HS20180716_1042-1045.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/Mar 2017 EDF files/0415_20170302_1707-1709.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Mar8 - 18Mar16/1074_MK20180309_1535-1440.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18July21-18July27/1400_LL20180724_1502-1504.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Aug27 - 18Aug31/1446_RT20180829_0858-0903.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr21 - 18Apr27/1208_LW20180426_1305-1310.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr21 - 18Apr27/1212_DW20180426_1250-1255.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr21 - 18Apr27/1184_GR20180426_1320-1325.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr21 - 18Apr27/1184_GR20180427_0958-1003.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr21 - 18Apr27/1206_20180427_1405-1410.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Mar23 - 18Mar30/1114_DM20180323_1335-1340.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Mar23 - 18Mar30/Theo2_old_20180328.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Mar31 - 18Apr6/1145_GP20180405_1005-1010.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Mar31 - 18Apr6/1145_GP20180406_1340-1345.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Mar31 - 18Apr6/1155_AD20180406_1405-1410.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/Aug 2017 EDF files/week of 17Aug25 to 17Aug31/0679_TS20170826_1038-1045.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/Aug 2017 EDF files/week of 17Aug4 to 17Aug10/0653_TF20170807_0938-0948.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 17Nov3 - 17Nov09/0823_CG20171102_1453-1457.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/Subjects 1-26/0017_20160308_1138.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 17Oct13 - 17Oct19/0794_FN20171019_0936-0940.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr28 - 18May4/1231_AB20180503_1210-1215.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr28 - 18May4/1233_MG20180504_1435-1440.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr14 - 18Apr20/1188_GV20180417_1000-1005.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr14 - 18Apr20/1185_TS20180419_1315-1320.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr14 - 18Apr20/1179_MH20180419_1410-1415.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr14 - 18Apr20/1177_SM20180419_1350-1355.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr14 - 18Apr20/1189_LH20180419_1330-1335.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr14 - 18Apr20/1184_GR20180420_1435-1440.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Apr14 - 18Apr20/1185_TS20180420_1530-1535.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/Sep 2017 EDF files/week of 15Sep17 to 17Sep21/0730_20170921_0947-0957.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Mar1 - 18Mar8/1052_DM20180303_1005-1010.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Mar1 - 18Mar8/1054_ML20180303_0945-0950.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Mar1 - 18Mar8/1063_KP20180308_1330-1335.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Mar1 - 18Mar8/1063_KP20180307_1400-1403.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Mar1 - 18Mar8/1067_JP20180308_1435-1440.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Aug5-18Aug11/1408_DG20180809_1532-1537.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/Feb - May 2016 EDF files/0017_20160308_1138.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Aug19 - 18Aug26/1433_RL20180819_0919-0924.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18June30-18July6/1346_TT20180704_1046-1048.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18June30-18July6/1346_TT20180705_1005-1006.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Jan18 - 18Feb15/0961_PW20180125_1401-1404.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Jan18 - 18Feb15/0991_RC20180208_1329-1333.edf', '/content/drive/MyDrive/EDF Files/EDF Files - Old EEG Device/week of 18Jan18 - 18Feb15/0991_RC20180209_1512-1515.edf']
  ctr = 1
  for cur_file in files:
    if cur_file in not_edf or cur_file in too_short:
      shitlist.append(cur_file)
      continue
    inputs = to_datetime(cur_file)
    if len(inputs) == 0:
      shitlist.append(cur_file)
      continue
    has_delerium = df.loc[(df['Date'] == inputs[1]) & (df['EEG STUDY ID'] == inputs[0]), 'Delirious (0=N, 1=Y)']
    if (len(has_delerium) == 0 or has_delerium.values[0] == '-' or has_delerium.values[0] == '?' or np.isnan(has_delerium.values[0])):
      shitlist.append(cur_file)
    else:
      filename_to_score[cur_file] = has_delerium.values[0]
  return filename_to_score, shitlist

In [ ]:
class FolderDataset(Dataset):
    def __init__(self, scores, files, spectrums, transform=None):
        self.folder_path = folder_path
        self.files = files
        self.transform = transform
        self.scores = scores
        self.spectrogram = spectrums

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = self.files[idx]
        spectrum = self.spectrogram[file_path]
        threshold = np.percentile(spectrum, 99.5)
        mask = spectrum > threshold
        spectrum[mask] = threshold
        spectrum = spectrum[:, :30, :100]
        score = self.scores[file_path]
        return spectrum, score

In [ ]:
from transformers import ViTImageProcessor
import torch

model_id = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTImageProcessor.from_pretrained(
    model_id
)

folder_path = '/content/drive/MyDrive/EDF Files'
batch_size = 32
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.5],
                             std=[0.5])
])

EDF_to_Scores, shitlist= csv_to_dict()
with open('/content/drive/MyDrive/filename_to_spec_psd_log.pkl', 'rb') as file:
    filename_to_spec = pickle.load(file)

files=get_list_of_files(folder_path)
files = [x for x in files if x not in shitlist]
dataset = FolderDataset(EDF_to_Scores, files, filename_to_spec, transform=transform)
num_samples = len(dataset)
print(num_samples)

train_size = int(0.7*num_samples)
test_size = int(0.15 * num_samples)
val_size = num_samples - train_size - test_size


train_dataset, test_dataset, val_dataset = random_split(dataset, [train_size, test_size, val_size])

#balance val dataset
subset_indices = val_dataset.indices
class_0_indices = [index for index in subset_indices if val_dataset.dataset[index][1] == 0]
class_1_indices = [index for index in subset_indices if val_dataset.dataset[index][1] == 1]
num_minority_examples = min(len(class_0_indices), len(class_1_indices))
num_majority_examples = max(len(class_0_indices), len(class_1_indices))
balanced_class_0_indices = random.sample(class_0_indices, num_minority_examples)
balanced_indices = balanced_class_0_indices + class_1_indices
val_dataset = torch.utils.data.Subset(val_dataset.dataset, balanced_indices)

minority = []
majority = []
max_vals = []
weights = []

subset_indices = train_dataset.indices
majority = [index for index in subset_indices if train_dataset.dataset[index][1] == 0]
minority = [index for index in subset_indices if train_dataset.dataset[index][1] == 1]

print(len(majority))
print(len(minority))

for idx in subset_indices:
  if train_dataset.dataset[idx][1] == 1:
      weights.append(0.5/len(minority))
  else:
      weights.append(0.5/len(majority))

sampler = WeightedRandomSampler(weights, num_samples=3000, replacement=True)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

3491
1854
586


In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(
        predictions=np.argmax(p.predictions, axis=1),
        references=p.label_ids
    )
num_ones = 0
num_zeros = 0
num_negs = 0
tot = 0
for batch in train_dataloader:
   num_ones = torch.sum(torch.eq(batch[1], 1.))
   num_zeros = torch.sum(torch.eq(batch[1], 0.))
   tot += batch[1].numel()
print(tot)
print('training set stats')

for batch in test_dataloader:
   num_ones = torch.sum(torch.eq(batch[1], 1).int())
   num_zeros = torch.sum(torch.eq(batch[1], 0.))
   tot += batch[1].numel()
print(tot)
print('test set stats')

<ipython-input-10-b4cf474e3bba>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


3000
training set stats
3523
test set stats


In [ ]:
def check_accuracy_part34(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)
            x = F.interpolate(x, size=(224, 224), mode='bilinear', align_corners=False)
            x = x.expand(-1, 3, -1, -1)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
            print(preds)
            print(y)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [ ]:
def train_model(model, optimizer, epochs=1):
    for e in range(epochs):
        for t, (x, y) in enumerate(train_dataloader):
            model.train()
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                print()

In [ ]:
def flatten(x):
    N = x.shape[0]
    return x.view(N, -1)

In [ ]:
from transformers import ViTFeatureExtractor
import torch

model_id = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(
    model_id
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
def preprocess(batch):
    inputs = feature_extractor(
        batch['img'],
        return_tensors='pt'
    )
    return inputs

prepared_train = train_dataset.with_transform(preprocess)
prepeared_test = test_dataset.with_transform(preprocess)

In [ ]:
from transformers import TrainingArguments
!pip install accelerate -U

training_args = TrainingArguments(
  output_dir="./output_dir",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  load_best_model_at_end=True,
)

In [ ]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained(
    model_id,
    num_labels=2
)

model.to(device)

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(
        predictions=np.argmax(p.predictions, axis=1),
        references=p.label_ids
    )

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=feature_extractor,
)

train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

In [ ]:
import numpy as np
import torch
import cv2
import torch.nn as nn
from transformers import ViTModel, ViTConfig
from torchvision import transforms
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm

model_checkpoint = 'google/vit-base-patch16-224-in21k'


class ViT(nn.Module):

  def __init__(self, config=ViTConfig(), num_labels=2,
               model_checkpoint='google/vit-base-patch16-224-in21k'):

        super(ViT, self).__init__()

        self.vit = ViTModel.from_pretrained(model_checkpoint, add_pooling_layer=False)
        self.classifier = (

            nn.Linear(config.hidden_size, num_labels)
        )

  def forward(self, x):

    x = self.vit(x)['last_hidden_state']
    output = self.classifier(x[:, 0, :])

    return output

In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics
from sklearn import decomposition
from sklearn.cluster import OPTICS

def get_f1(actual, predicted, num_classes):
  f1s = [];
  for i in range(num_classes):
    tp = 0;
    fp = 0;
    fn = 0;
    for j in range(len(predicted)):
      if(predicted[j] == actual[j] and actual[j] == i):
        tp += 1;
      if(predicted[j] == i and actual[j] != i):
        fp += 1;
      if(predicted[j] != i and actual[j] == i):
        fn += 1;
    precision = tp / (tp + fp + 1)
    recall = tp / (tp + fn + 1)
    f1 = 2*precision*recall / (precision+recall+1)
    f1s.append(f1)
  return f1s;

def get_confusion(actual, predicted):
    return metrics.confusion_matrix(actual, predicted)

def generate_figures(training_history, validation_history, confusion_matrix, clusters, labels):
    x = range(len(training_history))
    fig1 = plt.figure(1);
    print(type(training_history[0]))
    plt.plot(x, training_history);
    plt.xlabel("Epoch #")
    plt.ylabel("Cross Entropy Loss");
    plt.title("Training History");

    x = range(len(validation_history))
    fig2 = plt.figure(2);
    plt.plot(x, validation_history);
    plt.xlabel("Epoch #")
    plt.ylabel("Cross Entropy Loss");
    plt.title("Validation History");

    fig3 = plt.figure(3)
    ax = fig3.add_subplot(111)
    cax = ax.matshow(confusion_matrix, interpolation='nearest')
    fig3.colorbar(cax)

    plt.show()

    return fig1, fig2, fig3

In [ ]:
def check_accuracy(loader, model, val_history, dset):
    actual = []
    predicted = []
    if dset == 'val':
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')
    num_correct = 0
    num_samples = 0
    model.eval()
    val_loss = []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)

            x = F.interpolate(x, size=(224, 224), mode='bilinear', align_corners=False)
            x = x.expand(-1, 3, -1, -1)
            scores = model(x)
            if not isinstance(scores, torch.Tensor):
              scores = scores.logits
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)

            if dset == 'val':
              loss = F.cross_entropy(scores, y)
              val_loss.append(loss.cpu())
            if dset == 'test':
              actual = y.cpu()
              predicted = preds.cpu()
        acc = float(num_correct) / num_samples
        print(acc)
        if dset == 'val':
          val_history.append(sum(val_loss)/len(val_loss))
    return actual, predicted

In [ ]:
def model_train(epochs, learning_rate, bs):
    train_history = []
    val_history = []

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    model = ViT().to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = Adam(model.parameters(), lr=learning_rate)
    for e in range(epochs):
        total_acc_train = 0
        total_loss_train = 0.0
        for t, (x, y) in enumerate(train_dataloader):
            model.train()
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)
            x = F.interpolate(x, size=(224, 224), mode='bilinear', align_corners=False)
            x = x.expand(-1, 3, -1, -1)
            scores = model(x)
            loss = criterion(scores, y)
            acc = (scores.argmax(dim=1) == y.to(device)).sum().item()
            total_acc_train += acc
            total_loss_train += loss.item()

            loss.backward()
            optimizer.step()

            optimizer.zero_grad()
            if t % 10 == 0:
              print('cp')

        print(f"Epoch {e+1}/{epochs}")
        check_accuracy(val_dataloader, model, val_history, 'val')
        train_history.append(total_loss_train)
        print(f"Training Loss: {total_loss_train:.4f}")
    return train_history, val_history



EPOCHS = 5
LEARNING_RATE = 1e-4
BATCH_SIZE = 8

train_history, val_history = model_train(EPOCHS, LEARNING_RATE, BATCH_SIZE)
actual = []
predicted = []

with torch.no_grad():
  actual, predicted = check_accuracy(test_dataloader, model, val_history, 'test')
  f1 = get_f1(actual, predicted, 2)
  print("f1 = " + str(f1));
  confusion = get_confusion(actual, predicted)
  generate_figures(train_history, val_history, confusion, [], [])